<a href="https://colab.research.google.com/github/mathjams/AAAI_2024/blob/main/GRULSTMPPTPredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GRU/LSTM P/PT Predictions


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Embedding, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, GRU, Dense, TimeDistributed, Embedding, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
import statistics

Data Preparation


In [ ]:
def plainsequences(user_type):
  resulteye=[]
  resulthand=[]
  maxlen=0
  eye_basic_url='/Users/qyuvks/emily/AAAI/data_set/Eye_'
  hand_basic_url='/Users/qyuvks/emily/AAAI/data_set/Hand_'
  if (user_type=='ASD'):
    numOfUser=9
    eye_basic_url+="ASD_"
    hand_basic_url+='ASD_'
  else:
    eye_basic_url+="TD_"
    hand_basic_url+='TD_'
    numOfUser=17
  for i in range(1, numOfUser+1):
    for j in range(0,2):
      c_eye_url=eye_basic_url+'U'+str(i)+"_Active_"+str(j)+".xlsx"
      c_hand_url=hand_basic_url+'U'+str(i)+"_Active_"+str(j)+".xlsx"
      try:
        asd_eye_data=pd.read_excel(c_eye_url)
        asd_hand_data=pd.read_excel(c_hand_url)
        resulteye.append(asd_eye_data[['x','y']].to_numpy())
        resulthand.append(asd_hand_data[['x','y']].to_numpy())
        c_max_length=max(asd_eye_data.shape[0], asd_hand_data.shape[0])
        if c_max_length>maxlen:
          maxlen=c_max_length
      except IOError:
        print("")
  return resulteye, resulthand, maxlen

Data for Position and Time as input



In [ ]:
def plainsequences2(user_type):
  resulteye=[]
  resulthand=[]
  maxlen=0
  eye_basic_url='/Users/qyuvks/emily/AAAI/data_set/Eye_'
  hand_basic_url='/Users/qyuvks/emily/AAAI/data_set/Hand_'
  if (user_type=='ASD'):
    numOfUser=9
    eye_basic_url+="ASD_"
    hand_basic_url+='ASD_'
  else:
    eye_basic_url+="TD_"
    hand_basic_url+='TD_'
    numOfUser=17
  for i in range(1, numOfUser+1):
    for j in range(0,2):
      c_eye_url=eye_basic_url+'U'+str(i)+"_Active_"+str(j)+".xlsx"
      c_hand_url=hand_basic_url+'U'+str(i)+"_Active_"+str(j)+".xlsx"
      #asd_eye_data=pd.DataFrame()
      try:
        asd_eye_data=pd.read_excel(c_eye_url)
        asd_hand_data=pd.read_excel(c_hand_url)
        starttime= min(np.min(asd_hand_data['start']), np.min(asd_eye_data['start']))
        asd_eye_data['start']+=-starttime
        asd_eye_data['end']+=-starttime
        asd_hand_data['start']+=-starttime
        asd_hand_data['end']+=-starttime
        resulteye.append(asd_eye_data[['x','y','start', 'end']].to_numpy())
        resulthand.append(asd_hand_data[['x','y']].to_numpy())
        c_max_length=max(asd_eye_data.shape[0], asd_hand_data.shape[0])
        if c_max_length>maxlen:
          maxlen=c_max_length
      except IOError:
        print("")
  return resulteye, resulthand, maxlen

LSTM and GRU Models


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf

def masked_mse_loss(y_true, y_pred):
    # Create a mask to ignore the padded values (assumes padding is done with 0s)
    mask = tf.cast(tf.not_equal(y_true, 0), dtype=tf.float32)
    # Compute MSE and apply the mask
    mse = tf.square(y_true - y_pred) * mask
    # Take the mean of the MSE, dividing by the non-zero entries in the mask
    return tf.reduce_sum(mse) / tf.reduce_sum(mask)

def plot_loss(history):
    # Plot the training and validation loss over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.show()

def LSTM_model4(input_data, output_data, input_max_len, output_max_len, input_features, output_features, latent_dim, number_of_epochs):
    # Pad the input and output sequences to their respective maximum lengths
    input_data_padded = pad_sequences(input_data, maxlen=input_max_len, padding='post', dtype='float32', value=0)
    output_data_padded = pad_sequences(output_data, maxlen=output_max_len, padding='post', dtype='float32', value=0)

    # Shift the output sequences to create the target sequences
    decoder_target_data = np.roll(output_data_padded, shift=-1, axis=1)
    decoder_target_data[:, -1, :] = 0  # Reset last time step to 0 (zero padding)

    # Encoder
    encoder_inputs = Input(shape=(None, input_features), name='encoder_inputs')
    encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_LSTM')
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(None, output_features), name='decoder_inputs')
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_LSTM')
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = TimeDistributed(Dense(output_features, activation='linear'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn encoder_inputs and decoder_inputs into decoder_outputs
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=masked_mse_loss)

    # Fit the model
    result = model.fit(
        [input_data_padded, output_data_padded],
        decoder_target_data,
        batch_size=50,
        epochs=number_of_epochs,
        validation_split=0.2,
        verbose=1  # Set verbose=1 to print loss at each epoch
    )

    # Extract the validation loss from the history
    val_loss = result.history['val_loss']
    loss = result.history['loss']

    # Print the minimum validation loss
    min_val_loss = min(val_loss)
    print(f'Minimum validation loss: {min_val_loss}')

    # Plot the training and validation loss
    plot_loss(result.history)

    # Define the encoder model (used for encoding input sequences to their states)
    encoder_model = Model(encoder_inputs, encoder_states)

    # Define the decoder model (used for generating output sequences given the encoded states)
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    # Return model and training history
    return min_val_loss

def GRU_model4(input_data, output_data, input_max_len, output_max_len, input_features, output_features, latent_dim, number_of_epochs):
    # Pad the input and output sequences to their respective maximum lengths
    input_data_padded = pad_sequences(input_data, maxlen=input_max_len, padding='post', dtype='float32', value=0)
    output_data_padded = pad_sequences(output_data, maxlen=output_max_len, padding='post', dtype='float32', value=0)

    # Shift the output sequences to create the target sequences
    decoder_target_data = np.roll(output_data_padded, shift=-1, axis=1)
    decoder_target_data[:, -1, :] = 0  # Reset last time step to 0 (zero padding)

    # Encoder
    encoder_inputs = Input(shape=(None, input_features), name='encoder_inputs')
    encoder_gru = GRU(latent_dim, return_state=True, name='encoder_GRU')
    encoder_outputs, state_h = encoder_gru(encoder_inputs)
    encoder_states = [state_h]  # GRU only has a single hidden state

    # Decoder
    decoder_inputs = Input(shape=(None, output_features), name='decoder_inputs')
    decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True, name='decoder_GRU')
    decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)
    decoder_dense = TimeDistributed(Dense(output_features, activation='linear'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn encoder_inputs and decoder_inputs into decoder_outputs
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=masked_mse_loss)

    # Fit the model
    result = model.fit(
        [input_data_padded, output_data_padded],
        decoder_target_data,
        batch_size=50,
        epochs=number_of_epochs,
        validation_split=0.2,
        verbose=1  # Set verbose=1 to print loss at each epoch
    )

    # Extract the validation loss from the history
    val_loss = result.history['val_loss']
    loss = result.history['loss']

    # Print the minimum validation loss
    min_val_loss = min(val_loss)
    print(f'Minimum validation loss: {min_val_loss}')

    # Plot the training and validation loss
    plot_loss(result.history)

    # Define the encoder model (used for encoding input sequences to their states)
    encoder_model = Model(encoder_inputs, encoder_states)

    # Define the decoder model (used for generating output sequences given the encoded states)
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h]

    decoder_outputs, state_h_dec = decoder_gru(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    # Return model and training history
    return min_val_loss

Statistical Tests and Comparison of Models


In [ ]:
asdresulteye, asdresulthand = plainsequences('ASD')
tdresulteye, tdresulthand = plainsequences('TD')
asdresulteye2, asdresulthand2 =  plainsequences2('ASD')
tdresulteye2, tdresulthand2 =  plainsequences2('TD')

In [ ]:
#experiment on Active
import statistics
import scipy.stats as stats

asdgrufit2=[]
asdgrufit4=[]
asdlstmfit2 = []
asdlstmfit4 = []
tdlstmfit2=[]
tdlstmfit4=[]
tdgrufit2=[]
tdgrufit4=[]
for i in range(3):
    lval_losstd2 = LSTM_model4(tdresulteye, tdresulthand, 236, 236, 2, 2, 64, 1000)
    gval_losstd2 = GRU_model4(tdresulteye, tdresulthand, 236, 236, 2, 2, 64, 1000)
    tdlstmfit2.append(lval_losstd2)
    tdgrufit2.append(gval_losstd2)
    lval_losstd4 = LSTM_model4(tdresulteye2, tdresulthand2, 236, 236, 4, 2, 64, 1000)
    gval_losstd4 = GRU_model4(tdresulteye2, tdresulthand2, 236, 236, 4, 2, 64, 1000)
    tdlstmfit4.append(lval_losstd4)
    tdgrufit4.append(gval_losstd4)
    lval_lossasd2 = LSTM_model4(asdresulteye, asdresulthand, 260, 260, 2, 2, 64, 1000)
    gval_lossasd2 = GRU_model4(asdresulteye, asdresulthand, 260, 260, 2, 2, 64, 1000)
    asdlstmfit2.append(lval_lossasd2)
    asdgrufit2.append(gval_lossasd2)
    lval_lossasd4 = LSTM_model4(asdresulteye2, asdresulthand2, 260, 260, 4, 2, 64, 1000)
    gval_lossasd4 = GRU_model4(asdresulteye2, asdresulthand2, 260, 260, 4, 2, 64, 1000)
    asdlstmfit4.append(lval_lossasd4)
    asdgrufit4.append(gval_lossasd4)
print(asdlstmfit2)
print(asdlstmfit4)
print(asdgrufit2)
print(asdgrufit4)
print(tdlstmfit2)
print(tdlstmfit4)
print(tdgrufit2)
print(tdgrufit4)

In [ ]:
#experimental Results
print(statistics.mean(asdlstmfit2), statistics.mean(asdlstmfit4), statistics.mean(asdgrufit2), statistics.mean(asdgrufit4), statistics.mean(tdlstmfit2), statistics.mean(tdlstmfit4), statistics.mean(tdgrufit2), statistics.mean(tdgrufit4))
#compare within groups for ASD
asd2t_statistic, asd2p_value = stats.ttest_ind(asdlstmfit2, asdgrufit2)
print(f"The comparison between GRU and LSTM for ASD with 2 features is {asd2t_statistic} and {asd2p_value}")
asd4t_statistic, asd4p_value = stats.ttest_ind(asdlstmfit4, asdgrufit4)
print(f"The comparison between GRU and LSTM for ASD with 4 features is {asd4t_statistic} and {asd4p_value}")
l24asdt_statistic, l24asdp_value = stats.ttest_ind(asdlstmfit2, asdlstmfit4)
print(f"The comparison between 2 and 4 features in ASD with LSTM is {l24asdt_statistic} and {l24asdp_value}")
g24asdt_statistic, g24asdp_value = stats.ttest_ind(asdgrufit2, asdgrufit4)
print(f"The comparison between 2 and 4 features in ASD with GRU is {g24asdt_statistic} and {g24asdp_value}")
#compare within groups for TD
td2t_statistic, td2p_value = stats.ttest_ind(tdlstmfit2, tdgrufit2)
print(f"The comparison between GRU and LSTM for TD with 2 features is {td2t_statistic} and {td2p_value}")
td4t_statistic, td4p_value = stats.ttest_ind(tdlstmfit4, tdgrufit4)
print(f"The comparison between GRU and LSTM for TD with 4 features is {td4t_statistic} and {td4p_value}")
l24tdt_statistic, l24tdp_value = stats.ttest_ind(tdlstmfit2, tdlstmfit4)
print(f"The comparison between 2 and 4 features in TD with LSTM is {l24tdt_statistic} and {l24tdp_value}")
g24tdt_statistic, g24tdp_value = stats.ttest_ind(tdgrufit2, tdgrufit4)
print(f"The comparison between 2 and 4 features in TD with GRU is {g24tdt_statistic} and {g24tdp_value}")
#between group comparsion
l2t_statistic, l2p_value = stats.ttest_ind(asdlstmfit2, tdlstmfit2)
print(f"The comparison between ASD and TD for LSTM with 2 features is {l2t_statistic} and {l2p_value}")
g2t_statistic, g2p_value = stats.ttest_ind(asdgrufit2, tdgrufit2)
print(f"The comparison between ASD and TD for GRU with 2 features is {g2t_statistic} and {g2p_value}")
l4t_statistic, l4p_value = stats.ttest_ind(asdlstmfit4, tdlstmfit4)
print(f"The comparison between ASD and TD for LSTM with 4 features is {l4t_statistic} and {l4p_value}")
g4t_statistic, g4p_value = stats.ttest_ind(asdgrufit4, tdgrufit4)
print(f"The comparison between ASD and TD for GRU with 4 features is {g4t_statistic} and {g4p_value}")